In [2]:
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors
import pandas as pd
import math
from sklearn.linear_model import Lasso as Lasso_Reg
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score as val_score
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split as sk_split
import statsmodels.api as sm
import pycountry
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs

%matplotlib inline

In [3]:
#read in crops
allcrops_2000 = pd.read_csv('datasets/allcrops_2000.csv')
# names of all the crops
crop_names = allcrops_2000['ItemName'].unique()

# remove nan
crop_names = crop_names[:-1]
# list to put our separated crop dataframes
all_crop_dfs = []

# find each crop-separated df and put it in the list of dataframes
for crop in crop_names:
    crop_df = allcrops_2000[(allcrops_2000['ItemName'] == crop)]
    all_crop_dfs.append(crop_df)


In [4]:
# Do the same as above, but for livestock
livestock_2000 = pd.read_csv('datasets/livestock_2000.csv')
# list of livestock names
livestock_names = livestock_2000['ItemName'].unique()
livestock_names = livestock_names[:-1]

# list to put our separated dataframes
livestock_dfs = []

# separate dataframe based on itemname
for crop in livestock_names:
    crop_df = livestock_2000[(livestock_2000['ItemName'] == crop)]
    livestock_dfs.append(crop_df)

print livestock_names


['Population' 'Bovine Meat' 'Mutton & Goat Meat' 'Pigmeat' 'Poultry Meat'
 'Meat, Other' 'Offals, Edible' 'Fats, Animals, Raw' 'Milk, Whole'
 'Butter, Ghee' 'Cheese' 'Cream' 'Eggs' 'Honey' 'Freshwater Fish'
 'Demersal Fish' 'Pelagic Fish' 'Marine Fish, Other' 'Crustaceans'
 'Cephalopods' 'Molluscs, Other' 'Aquatic Plants' 'Fish, Body Oil'
 'Fish, Liver Oil' 'Milk - Excluding Butter' 'Meat' 'Offals' 'Animal fats'
 'Fish, Seafood' 'Aquatic Products, Other' 'Grand Total' 'Animal Products'
 'Aquatic Animals, Others' 'Whey' 'Meat Meal' 'Meat, Aquatic Mammals'
 'Hides and skins']


In [5]:
# read in disease rates
column_headers = ["Country", "Year", "Mortality", "Mortality 2", "Mortality 3"]
diabetes = pd.read_csv('datasets/diabetes.csv', names = column_headers)
cardio = pd.read_csv('datasets/cardiovascular.csv', names = column_headers)
cancer= pd.read_csv('datasets/Cancer.csv', names = column_headers)

In [6]:
cardio.iloc[3:, 0:3].head(n=5)
#cardio.columns = ['Country', 'Year', 'Mortality']

,Country,Year,Mortality
3,Afghanistan,2012,511.5
4,Afghanistan,2000,542.3
5,Albania,2012,436.2
6,Albania,2000,560.5
7,Algeria,2012,396.6


In [26]:
cardio_2012 = cardio[(cardio["Year"] == "2012")] 
cardio_2000 = cardio[(cardio["Year"] == "2000")] 
cardio_2000.head(n=5)

,Country,Year,Mortality,Mortality 2,Mortality 3
4,Afghanistan,2000,542.3,554.1,523.1
6,Albania,2000,560.5,494.7,638.6
8,Algeria,2000,434.7,401.2,472.2
10,Angola,2000,383.4,385.2,375.2
12,Argentina,2000,226.6,169.1,302.8


In [8]:
diabetes_2012 = diabetes[(diabetes['Year'] == '2012')] 
diabetes_2000 = diabetes[(diabetes['Year'] == '2000')] 

In [11]:
cancer_2012 = cancer[(cancer['Year'] == ' 2012')] 
cancer_2000 = cancer[(cancer['Year'] == ' 2000')] 
cancer_2000.head(n=5)

,Country,Year,Mortality,Mortality 2,Mortality 3
4,Afghanistan,2000,122.9,107.9,143.6
6,Albania,2000,137.7,109.0,169.4
8,Algeria,2000,75.1,77.8,72.6
10,Angola,2000,74.8,73.4,80.3
12,Argentina,2000,136.8,112.1,173.3


In [17]:
# obtain all countries with crop data
country_names = cardio['Country'].unique()
country_names = country_names[:-1]
country_names

array([nan, 'Country', 'Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', "Cote d'Ivoire", 'Cabo Verde',
       'Cambodia', 'Cameroon', 'Canada', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Fiji',
       'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany',
       'Ghana', 'Greece', 'Guatem

In [84]:
cardio_new = cardio_2000[['Country', 'Mortality']]
def serbia():
    for index, country in enumerate(cardio_new["Country"]):
        if country == "Serbia":
            serbia_mort = cardio_new.iloc[index, :]
            return serbia_mort
def montenegro():
    for index, country in enumerate(cardio_new["Country"]):
        if country == "Montenegro":
            montenegro_mort = cardio_new.iloc[index, :]
            return montenegro_mort

serbia_mort = serbia()
serbia_mort =  float(serbia_mort[1])
montenegro_mort = montenegro()
montenegro_mort = float(montenegro_mort[1])
serbia_mont = (serbia_mort +  montenegro_mort)/2
print serbia_mont

438.3


In [31]:
input_countries = country_names

counter = 1
for each in range(1, len(input_countries)):
    if input_countries[counter] == 'The former Yugoslav republic of Macedonia':
        input_countries[counter] = 'The former Yugoslav Republic of Macedonia'
    if input_countries[counter] == 'United Kingdom of Great Britain and Northern Ireland':
        input_countries[counter] = 'United Kingdom'
    if input_countries[counter] == "Sudan":
        input_countries[counter] = 'Sudan (former)'
    if input_countries[counter] == 'Czech Republic':
        input_countries[counter] = 'Czechia'
    if input_countries[counter] == 'Serbia':
        input_countries[counter] = 'Serbia and Montenegro'
    counter = counter + 1

In [19]:
input_countries

array([nan, 'Country', 'Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', "Cote d'Ivoire", 'Cabo Verde',
       'Cambodia', 'Cameroon', 'Canada', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Fiji',
       'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany',
       'Ghana', 'Greece', 'Guatemala', '